<img src="../gfx/euro2.png" alt="Euro" style="display: block; margin: auto; width: 100%;" />

**ONCFM - Organisation nationale de lutte contre le faux-monnayage** est une organisation publique ayant pour objectif de mettre en place des méthodes d'identification des contrefaçons des billets en euros.
Dans le cadre de cette lutte, nous souhaitons mettre en place un algorithme qui soit capable de différencier automatiquement les vrais des faux billets.

<div style="text-align:center;"><h1>Détecteur de faux billets - Application</h1></div>

## Objectif

Dans le premier notebook, nous avons entraîné un modèle de machine learning à détecter les faux billets suivant leurs dimensions.

Il s'agit maintenant de mettre en production ce modèle avec des nouveaux jeux de données.

<img src="../gfx/sep.jpg" alt="Barre">

## Chargement

***

In [1]:
import pandas as pd
import seaborn as sns
import joblib

sns.color_palette("viridis")
sns.set_theme(style="darkgrid")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import _base

In [2]:
# Chemin du modèle
le_modele_path = 'le_modele_1.0.pkl'

<img src="../gfx/sep.jpg" alt="Barre">

## 1 - Chargement du fichier de données

In [3]:
data = pd.read_csv("../data/billets_production.csv")
data.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


## 2 - Application du pipeline

> Traitement de la conformité des variables.

> Traitement des données manquantes par régression linéaire.

> Standardisatiion des données.

> Application du modèle pré-entraîné.

In [4]:
class ColumnConformityTransformer():
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        print("Traitement du jeu de données...")
        print("===============================")
        return self
    
    def transform(self, X):
        # Vérifier si les colonnes sont présentes dans le DataFrame
        missing_columns = set(self.columns).difference(X.columns)
        if not missing_columns:
            print("Le jeu de données ne comporte aucune colonne manquante.")
            print("=======================================================")
            return X
        else:
            print("Le jeu de données comporte des colonnes manquantes.")
            print("===================================================")
            raise ValueError(f"Colonnes manquantes : {missing_columns}")
        
class MissingValuesRegressor(_base.TransformerMixin):
    def __init__(self, columns=None):
        super().__init__()
        self.columns = columns

    def fit(self, X, y=None):
        # Si des colonnes sont spécifiées, on les filtre
        if self.columns is not None:
            X = X[self.columns]

        # Entraînement d'un modèle de régression linéaire pour chaque colonne dave des valeurs manquantes
        self.models = {}
        for col in X.columns:
            mask = X[col].isna()
            if mask.any():
                print(f"Colonne {col} comporte des valeurs manquantes.")
                model = LinearRegression()
                X_train = X[~mask]
                y_train = X_train[col]
                model.fit(X_train.drop(col, axis=1), y_train)
                self.models[col] = model
            else:
                print(f"Colonne {col} ne comporte aucune valeur manquante.")
        return self

    def transform(self, X):
        # Si des colonnes sont spécifiées, on les filtre
        if self.columns is not None:
            X = X[self.columns]

        # Prédiction des valeurs manquantes pour chaque colonne
        X_transformed = X.copy()
        for col in X_transformed.columns:
            mask = X_transformed[col].isna()
            if mask.any() and col in self.models:
                # Prédire les valeures manquantes
                X_filled = self.models[col].predict(X_transformed[~mask].drop(col, axis=1))
                X_transformed.loc[mask, col] = X_filled
                print(f"Colonne {col} : {mask.sum()} valeurs manquantes remplacées.")
        return X_transformed
    
class TrainedModelApplier(_base.TransformerMixin):
    def __init__(self, model_path):
        self.model_path = model_path

    def fit(self, X, y=None):
        print("=======================================================")
        print("Chargement du modèle déjà entraîné...")
        print("=====================================")
        # Chargement du modèle déjà entraîné
        self.model = joblib.load(self.model_path)
        return self
        
    def transform(self, X):
        # Appliquer le modèle pour détecter les faux billets
        predictions = self.model.predict(X)
        print("Fin du traitement des données.")
        
        # Retourner les prédictions
        return pd.DataFrame(
            {
                'ID': data['id'],
                "Prédiction de l'authenticité": predictions,
                "Probabilité Vrai": self.model.predict_proba(X)[:, 1],
                "Probabilité Faux": self.model.predict_proba(X)[:, 0]
            }
        )

In [5]:
# Définition des colonnes attendues
colonnes = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']

In [6]:
# Créer le pipeline
pipeline = Pipeline([
    ('column_checker', ColumnConformityTransformer(columns=colonnes)),
    ('missing_imputer', MissingValuesRegressor(columns=colonnes)),
    ('scaler', StandardScaler()),
    ('trained_model', TrainedModelApplier(le_modele_path))
])
pipeline

Pipeline(steps=[('column_checker',
                 <__main__.ColumnConformityTransformer object at 0x15b730c50>),
                ('missing_imputer',
                 <__main__.MissingValuesRegressor object at 0x15b732960>),
                ('scaler', StandardScaler()),
                ('trained_model',
                 <__main__.TrainedModelApplier object at 0x15b730380>)])

## 3 - Résultats des prédictions

In [7]:
# Formattage des résultats
def genuine_condition(v):
    if v == 0:
        return 'Faux billet'
    else:
        return 'Vrai billet'
    
def format_genuine(styler):
    styler.set_caption("Analyse de l'authenticité des billets")
    styler.set_properties(
        subset=["Prédiction de l'authenticité"],
        background='green'
    ),
    styler.highlight_min(
        subset=["Prédiction de l'authenticité"],
        color='red'
    ),
    styler.format(
        subset=["Prédiction de l'authenticité"],
        formatter=genuine_condition
    )
    return styler

In [8]:
pred = pipeline.fit_transform(data)
pred.style.pipe(format_genuine)

Traitement du jeu de données...
Le jeu de données ne comporte aucune colonne manquante.
Colonne diagonal ne comporte aucune valeur manquante.
Colonne height_left ne comporte aucune valeur manquante.
Colonne height_right ne comporte aucune valeur manquante.
Colonne margin_low ne comporte aucune valeur manquante.
Colonne margin_up ne comporte aucune valeur manquante.
Colonne length ne comporte aucune valeur manquante.
Chargement du modèle déjà entraîné...
Fin du traitement des données.


,ID,Prédiction de l'authenticité,Probabilité Vrai,Probabilité Faux
0,A_1,Faux billet,0.082850,0.917150
1,A_2,Faux billet,0.083494,0.916506
2,A_3,Faux billet,0.015797,0.984203
3,A_4,Vrai billet,0.986041,0.013959
4,A_5,Vrai billet,0.998316,0.001684


 <img src="../gfx/sep.jpg" alt="Barre">